**Passo 1: Configuração do Ambiente no Google Colab**
Primeiro, preparamos o ambiente no Colab, habilitando a GPU e conectando ao seu Google Drive para acessar os dados.

In [15]:
# Mount Google Drive to access our dataset
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Passo 2: Preparação do Dataset**<BR>
Agora vamos carregar e pré-processar suas imagens. As transformações definidas são padrão para modelos pré-treinados no ImageNet.

In [16]:
import torch
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader

# 1. Define the image transformations
# Images will be resized, cropped, converted to tensors, and normalized.
# Normalization uses the standard mean and std dev from ImageNet models.
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 2. Set the path to your train and validation folders on Google Drive
# ATTENTION: Adjust this path to where you saved your dataset!
train_path = '/content/drive/MyDrive/DEV/dataset_pizza_burger/training'
validation_path = '/content/drive/MyDrive/DEV/dataset_pizza_burger/validation'

# 3. Load the datasets using ImageFolder
train_dataset = datasets.ImageFolder(train_path, transform=transform)
validation_dataset = datasets.ImageFolder(validation_path, transform=transform)

# 4. Create the DataLoaders
# They are responsible for loading the data in batches.
trainloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valloader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# Check the classes that were found (should be ['burger', 'pizza'])
print("Classes found:", train_dataset.classes)

Classes found: ['burger', 'pizza']


**Passo 3: Escolha e Carregamento do Modelo Pré-treinado**

Vamos usar a ResNet18, um modelo famoso e eficiente que já foi treinado no dataset ImageNet.

In [17]:
# Load a pretrained ResNet18 model
model = models.resnet18(pretrained=True)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


**Passo 4: Adaptação do Modelo para o Nosso Problema**

Esta é a etapa central do Transfer Learning. Primeiro, "congelamos" as camadas que já foram treinadas para não perdermos o conhecimento adquirido. Depois, substituímos a camada de classificação final para que ela se ajuste ao nosso problema de 2 classes.

In [18]:
from torch import nn

# 1. Freeze the parameters of the pretrained model
for param in model.parameters():
    param.requires_grad = False

# 2. Replace the final layer (the classifier)
# The original classifier of ResNet18 is named 'fc' (fully connected).
# It has 512 input features and 1000 output features (for ImageNet).
# We will replace it with a new one with 512 inputs and 2 outputs for our classes.
n_inputs = model.fc.in_features
model.fc = nn.Linear(n_inputs, 2) # 2 classes: pizza and burger

**Passo 5: Definição do Otimizador e da Função de Perda**

Configuramos as ferramentas para o novo treinamento. É importante notar que passamos para o otimizador apenas os parâmetros da nova camada que criamos, pois são os únicos que precisam ser treinados.

In [19]:
from torch import optim

# Loss Function - CrossEntropyLoss is standard for classification
criterion = nn.CrossEntropyLoss()

# Optimizer - Adam is a robust choice
# IMPORTANT: We only pass the parameters of the new final layer, as these are the only ones that need to be trained.
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

**Passo 6: Treinamento e Validação do Modelo**

Agora executamos o loop de treinamento. Este código já inclui a fase de validação para monitorar o desempenho do modelo em dados que ele não viu durante o treino.

In [20]:
import time

def train_and_validate(model, trainloader, valloader, criterion, optimizer, device, epochs=5): # <- 5 epochs
    """
    Function to train and validate a PyTorch model.
    """
    print("Starting training...")
    start_time = time.time()

    for epoch in range(epochs):
        # --- Training Phase ---
        model.train()  # Set model to training mode
        train_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # --- Validation Phase ---
        model.eval()  # Set model to evaluation mode
        validation_loss = 0.0
        accuracy = 0
        with torch.no_grad(): # Disable gradient calculation for validation
            for images, labels in valloader:
                images, labels = images.to(device), labels.to(device)
                output = model(images)
                validation_loss += criterion(output, labels).item()

                # Calculate accuracy
                ps = torch.exp(output)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {train_loss/len(trainloader):.4f} | "
              f"Validation Loss: {validation_loss/len(valloader):.4f} | "
              f"Validation Accuracy: {accuracy/len(valloader)*100:.2f}%")

    total_time = (time.time() - start_time) / 60
    print(f"\nTraining finished in {total_time:.2f} minutes.")

# Start the training!
history = train_and_validate(model, trainloader, valloader, criterion, optimizer, device, epochs=5)

Starting training...


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch 1/5 | Train Loss: 0.8169 | Validation Loss: 0.6813 | Validation Accuracy: 60.59%
Epoch 2/5 | Train Loss: 0.5517 | Validation Loss: 0.5328 | Validation Accuracy: 74.83%
Epoch 3/5 | Train Loss: 0.4047 | Validation Loss: 0.3290 | Validation Accuracy: 88.19%
Epoch 4/5 | Train Loss: 0.3163 | Validation Loss: 0.2929 | Validation Accuracy: 95.31%
Epoch 5/5 | Train Loss: 0.2542 | Validation Loss: 0.2575 | Validation Accuracy: 95.31%

Training finished in 2.17 minutes.


**O que observar na saída acima:**

`Train Loss` (Perda de Treino): Mede o erro do modelo nos dados de treino. O esperado é que este número diminua a cada época, mostrando que o modelo está aprendendo.

`Validation Loss` (Perda de Validação): Mede o erro nos dados que o modelo nunca viu (validação). Este é o indicador mais importante! O esperado é que ele também diminua. Se ele começar a subir enquanto a `Train Loss` continua caindo, é um sinal de overfitting.

`Validation Accuracy` (Acurácia de Validação): A porcentagem de acertos nos dados de validação. O esperado é que este número aumente, chegando o mais perto possível de 100%.

Após o término de todas as épocas, a última linha a ser impressa será o tempo total do processo.

# O "Produto" Final: O Modelo Treinado
Este é o resultado mais importante do projeto. Ao final do processo, a variável `model` não é mais um modelo genérico; ela agora contém os pesos ajustados e o conhecimento específico para diferenciar pizzas e hambúrgueres.

Você pode usar este modelo treinado para fazer previsões em imagens novas que ele nunca viu antes. Por exemplo, você poderia adicionar uma nova célula no Colab para testar uma imagem da internet:

**Passo 7: Função de Predição**

Esta é a função para fazer previsões em novas imagens da internet

In [24]:
# --- CELL 7: Prediction Function (FINAL CORRECTED VERSION) ---

from PIL import Image
import requests
import io
import torch

def predict_image(image_url):
    """
    Function to load an image from a URL and predict its class.
    This version includes a User-Agent header to avoid blocking issues.
    """
    try:
        # Define a browser-like header
        headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'}

        # Get the image data from the URL using the header
        response = requests.get(image_url, headers=headers)

        # Check if the request was successful
        response.raise_for_status()

        # Convert the byte data into a file-like object
        image_bytes = io.BytesIO(response.content)

        # Open the image
        image = Image.open(image_bytes).convert('RGB')

    except requests.exceptions.HTTPError as errh:
        print(f"Http Error: {errh}")
        return
    except Exception as e:
        print(f"Error loading image: {e}")
        return

    # Apply the same transformations as the training set
    transformed_image = transform(image).unsqueeze(0)

    model.eval() # Set the model to evaluation mode
    with torch.no_grad(): # Disable gradient calculation
        output = model(transformed_image.to(device))
        ps = torch.exp(output)
        top_p, top_class = ps.topk(1, dim=1)
        prediction_idx = top_class.item()

    classes = train_dataset.classes
    prediction = classes[prediction_idx]
    probability = top_p.item() * 100

    print(f"Prediction: The image is a: {prediction}")
    print(f"Probability: {probability:.2f}%")

**Passo 8: Testar o Modelo Treinado**

Finalmente, vamos testar nosso modelo com algumas imagens que ele nunca viu antes!

In [26]:
# --- CELL 8: Test the Trained Model ---

# Example 1: Hamburger
hamburger_url = "https://i.imgur.com/YLPiFZh.jpeg"
print(f"Testing image from: {hamburger_url}")
predict_image(hamburger_url)

print("-" * 30)

# Example 2: Pizza
pizza_url = "https://i.imgur.com/eeDDFhu.jpeg"
print(f"Testing image from: {pizza_url}")
predict_image(pizza_url)

Testing image from: https://i.imgur.com/YLPiFZh.jpeg
Prediction: The image is a: burger
Probability: 671.37%
------------------------------
Testing image from: https://i.imgur.com/eeDDFhu.jpeg
Prediction: The image is a: pizza
Probability: 671.93%
